In [5]:

# code file
import pandas as pandas
import numpy as np
from bs4 import BeautifulSoup as bs
import urllib.request,sys,time
import requests

In [7]:
from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd

#googlenews = GoogleNews(lang='en', region='US')

googlenews = GoogleNews(start='01/01/2013', end='04/04/2023')
googlenews.search('Housing Ireland')
result = googlenews.result(sort=True)
df = pd.DataFrame.from_dict(result)
print(googlenews.total_count())
titles = googlenews.get_texts()

HTTP Error 429: Too Many Requests
0
